In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os.path
import access_data 
import preprocessing
import setting
import feature
import sys
import time

pd.set_option('display.max_columns' ,1000)
pd.set_option('display.max_rows',60)
%matplotlib inline

#清理 + 分词 + label encode+计算词频

In [2]:
train_corpus ,test_corpus = preprocessing.clean_corpus()
test_corpus['corpus'] = preprocessing.segment_word(test_corpus['corpus'])
train_corpus['corpus'] = preprocessing.segment_word(train_corpus['corpus'])

train_corpus.to_pickle(setting.processed_data_dir+'cleaned&segment_train')
test_corpus.to_pickle(setting.processed_data_dir+'cleaned&segment_test')

train_uid,test_uid = preprocessing.encode_label()

preprocessing.bag_of_word(train_corpus['corpus'].values,test_corpus['corpus'].values,min_df=10)

INFO:root:finished cleaning symbol http[0-9a-zA-Z?:=._@%/\-#&\+|]+
INFO:root:finished cleaning symbol //@
INFO:root:finished cleaning symbol @
INFO:root:finished cleaning symbol #
INFO:root:finished cleaning symbol 【
INFO:root:finished cleaning symbol 《
INFO:root:finished cleaning symbol \[
INFO:root:Start to segment
Building prefix dict from C:\Anaconda\lib\site-packages\jieba\dict.txt ...
DEBUG:jieba:Building prefix dict from C:\Anaconda\lib\site-packages\jieba\dict.txt ...
Loading model from cache c:\users\admini~1\appdata\local\temp\jieba.cache
DEBUG:jieba:Loading model from cache c:\users\admini~1\appdata\local\temp\jieba.cache
Loading model cost 0.955 seconds.
DEBUG:jieba:Loading model cost 0.955 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
INFO:root:have segmented 0
INFO:root:have segmented 100000
INFO:root:Start to segment
INFO:root:have segmented 0
INFO:root:have segmented 100000
INFO:root:have segmented 200000
INFO:root:

##主题模型

In [ ]:
%run run_lda.py

  ##构建用户基本特征 

In [8]:
train_uid = access_data.load_processed_data('uid&pid_train')
test_uid  = access_data.load_processed_data('uid&pid_test')

train ,test = access_data.load_raw_data()

train_corpus = access_data.load_processed_data('cleaned&segment_train')
test_corpus  = access_data.load_processed_data('cleaned&segment_test')                                               
train = pd.concat([train_uid,train,train_corpus],axis=1)
test  = pd.concat([test_uid,test,test_corpus],axis=1)

train.drop([0,1],axis=1,inplace=True)
test.drop([0,1],axis=1,inplace=True)

train.columns = ['pid','uid','time','share','comment','zan','raw_corpus','clean&segment','链接','//@','@','#','【','《','\[']
test.columns = ['pid','uid','time','raw_corpus','clean&segment','链接','//@','@','#','【','《','\[']
train['uid'] = train['uid'].astype(np.uint16)
test['uid']  = test['uid'].astype(np.uint16)
l = ['链接','//@','@','#','【','《','\[']

for string in l :
    train[string] = train[string].astype(np.int8)
    test[string]  = test[string].astype(np.int8)

#在training set和test set中和用户发送微博的总数量
tot = pd.concat([pd.DataFrame(train['uid']),pd.DataFrame(test['uid'])])
c = pd.DataFrame(tot['uid'].value_counts())
c.columns = ['tot_counts']
train = train.merge(c,left_on='uid',right_index=True,how='left')
test  = test.merge(c,left_on='uid',right_index=True,how='left')

# 用户出现在训练集的次数
c = pd.DataFrame(train['uid'].value_counts())
c.columns = ['train_counts']
train = train.merge(c,left_on='uid',right_index=True,how='left')
test  = test.merge(c,left_on='uid',right_index=True,how='left')

test.fillna(-1,inplace=True)
train['tot_counts'] = train['tot_counts'].astype(np.int32)
train['train_counts'] = train['train_counts'].astype(np.int32)

test['tot_counts'] = test['tot_counts'].astype(np.int32)
test['train_counts'] = test['train_counts'].astype(np.int32)

addr1 = setting.raw_data_dir + 'basic_train'
addr2 = setting.raw_data_dir + 'basic_test'

lda_result = np.load('processed_data/lda_result_version3.npy')
lda_result = pd.DataFrame(lda_result,columns=['topic_%d' %i for i in range(0,25)])

for string in ['topic_%d' %i for i in range(0,25)]:
    train[string] = lda_result.loc[:train.shape[0]-1,string].values
    test[string] = lda_result.loc[train.shape[0]:,string].values

#train.to_pickle(addr1)
#test.to_pickle(addr2)

##用户特征 + 时间特征  + 文本特征

In [2]:
#train_basic = pd.read_pickle('raw_data/basic_train')
#test_basic  = pd.read_pickle('raw_data/basic_test')
#train_basic = train_basic.loc[1626750:]
#计算情感极性
begin_time= time.time()
train_sentiment,test_sentiment = feature.sentiment_feature(train_basic,test_basic)
end_time = time.time()
print end_time - begin_time
#计算一周内出现微博的数量
begin_time= time.time()
train_seven_days ,test_seven_days = feature.find_seven_days(train_basic,test_basic)
end_time = time.time()
print end_time - begin_time
#lda特征
begin_time = time.time()
train_lda_feature,test_lda_feature = feature.lda_feature(train_basic,test_basic)
end_time = time.time()
print end_time - begin_time
#用户特征
begin_time= time.time()
train_user,test_user = feature.user_basic_feature(train_basic,test_basic)
end_time = time.time()
print end_time - begin_time
#文本特征
begin_time= time.time()
train_content,test_content = feature.content_basic_feature(train_basic,test_basic)
end_time = time.time()
print end_time - begin_time
#时间特征
begin_time= time.time()
train_time,test_time = feature.time_feature(train_basic,test_basic)
end_time = time.time()
print end_time - begin_time
#关键词特征
begin_time = time.time()
train_keyword,test_keyword = feature.key_word_feature(train_basic,test_basic)
end_time = time.time()
print end_time - begin_time

In [ ]:
train_basic.drop(['uid','pid','time','share','comment','zan','raw_corpus','clean&segment'],axis=1,inplace=True)
test_basic.drop(['uid','pid','time','raw_corpus','clean&segment'],axis=1,inplace=True)
train_user = pd.concat([train_user,train_basic],axis=1)
test_user  = pd.concat([test_user,test_basic],axis=1)

train = train_user.merge(train_content,how='left',left_on='pid',right_index=True)
test  = test_user.merge(test_content,how='left',left_on='pid',right_index=True)

train = train.merge(train_time,how='left',left_on='pid',right_index=True)
test  = test.merge(test_time,how='left',left_on='pid',right_index=True)

train = pd.concat([train,train_keyword],axis=1)
test  = pd.concat([test,test_keyword],axis=1)

train = train.merge(train_lda_feature,how='left',left_on='pid',right_index=True)
test  = test.merge(test_lda_feature,how='left',left_on='pid',right_index=True)

train = train.merge(train_sentiment,how='left',left_on='pid',right_index=True)
test  = test.merge(test_sentiment,how='left',left_on='pid',right_index=True)

train = train.merge(train_seven_days,how='left',left_on='pid',right_index=True)
test  = test.merge(test_seven_days,how='left',left_on='pid',right_index=True)

begin_time = time.time()
result,gmm = feature.clustering_feature(train,test)
end_time = time.time()
print end_time - begin_time

train = train.merge(result,how='left',left_on='uid',right_index=True)
test  = test.merge(result,how='left',left_on='uid',right_index=True)

##整理格式

In [3]:
#train.drop(['sentiment','seven_days'],axis=1,inplace=True)
#test.drop(['sentiment','seven_days'],axis=1,inplace=True)
result_test = []
result_train = []
tot = 0
for string in ['share','comment','zan','content_len','链接','//@','@','#','【','《','\[']:
    temp = []
    for i in test[string+'_histogram']:
        if isinstance(i,int):
            temp.append(np.zeros(shape=8))
            tot +=1
        else:
            temp.append(i[0])
    result_test.append(np.asarray(temp))
    temp = []
    for i in train[string+'_histogram']:
        temp.append(i[0])
    result_train.append(np.asarray(temp))
    
    train.drop(string+'_histogram',axis=1,inplace=True)
    test.drop(string+'_histogram',axis=1,inplace=True)
train.drop(['pid','uid'],inplace=True,axis = 1)
test.drop(['pid','uid'],inplace=True,axis = 1)

train_y = train[['share','comment','zan']].values
train.drop(['share','comment','zan'],axis = 1,inplace=True)
train_x = train.values
test_x  = test.values
for i in result_train:
    train_x = np.c_[train_x,i]
for i in result_test:
    test_x = np.c_[test_x,i]
np.save('processed_data/train3_np',train_x)
np.save('processed_data/test3_np',test_x)
np.save('processed_data/target3_np',train_y)

In [ ]:
%run learning.py